# EmpkinS IO

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg
import neurokit2 as nk

import re

from datetime import datetime

import biopsykit as bp
from biopsykit.signals.ecg import EcgProcessor

from empkins_io.sensors.radar import load_radar_data, load_radar_data_raw

import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

pg.options['round'] = 3

palette

In [ ]:
subject_id = "VP_06"
condition = "stress"

In [ ]:
base_path = Path("../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy")
data_path = base_path.joinpath("data_per_subject/{}/{}".format(subject_id, condition))

ecg_path = data_path.joinpath("ecg/raw")
radar_path = data_path.joinpath("radar/raw")
video_path = data_path.joinpath("video")
timelog_path = data_path.joinpath("time_log/processed")

In [ ]:
ecg_files = list(sorted(ecg_path.glob("*.bin")))

In [ ]:
radar_files = list(sorted(radar_path.glob("*.mat")))
#radar_files

In [ ]:
index = 1

In [ ]:
ecg, fs_ecg = bp.io.nilspod.load_dataset_nilspod(ecg_files[index])

In [ ]:
radar_raw, fs_radar = load_radar_data_raw(radar_files[index])
radar, fs_radar = load_radar_data(radar_files[index], datastreams=["hr", "resp"])
radar = radar["hr"]

In [ ]:
import matplotlib
fig, ax = plt.subplots()
ecg[["acc_x", "acc_y", "acc_z"]].plot(ax=ax)
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%H:%M:%S.%f"))

In [ ]:
#ecg[["ecg"]].plot()

In [ ]:
#radar_raw[["pulse"]].plot()

In [ ]:
#radar_raw[["resp"]].plot()

In [ ]:
ep = EcgProcessor(ecg, fs_ecg)
ep.ecg_process()

hr_ecg = ep.heart_rate["Data"]

In [ ]:
hr_radar = radar[["Heart_Rate"]]

In [ ]:
bp.signals.ecg.plotting.ecg_plot(ecg_processor=ep, key="Data", figsize=(10,5))

In [ ]:
fig, ax = plt.subplots()

hr_ecg.plot(ax=ax, legend="ECG")
hr_radar.plot(ax=ax, legend="Radar")

In [ ]:
from ipywidgets import Video, Audio

In [ ]:
video_file = video_path.joinpath("VP_06_EB_pre.mp4")

In [ ]:
Video.from_file(video_file)

In [ ]:
import librosa
from librosa.display import waveplot
import ffmpeg

In [ ]:
probe = ffmpeg.probe(video_file)

In [ ]:
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
video_info

In [ ]:
audio = ffmpeg.input(video_file).audio

In [ ]:
out = ffmpeg.output(audio, filename=video_path.joinpath("test.mp3"))
out.run()

In [ ]:
audio, fs = librosa.load(video_path.joinpath("test.mp3"))

In [ ]:
fig, ax = plt.subplots()
waveplot(audio)